# Problem
This project is a solution to the hackathon of Kin Analytics company. The problem is to predict which customers will drop a product in less than two years based on their use of the company's products and financial data such as bank account transactions and credit bureau scores.

## Data
The data to work are four datasets with information (Data aren't in GitHub repository because the size is too big. Instead, data are in a folder on google drive.): 
1) clients: This is a historical database of clients that have purchased the product “Kin Safety” from 2012 to the delivery date.
2) clientProducts: Historical of the products the client had with the Kin Security at the moment he/she applied to the new product.
3) transactions: The transactions of clients in the pre-application moment.
4) creditScore: Bureau score of clients 6 months before and 6 months after the client applied to the new product.

In [1]:
# All packages to work in the problem
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
# Load all datasets, This process may take a while due to the download process.
clients = pd.read_csv("https://drive.google.com/uc?&confirm=no_antivirus&id=1JdAGtUh4lD4yUCDh5Og5mz8Laa1tBwFH")
clientProducts = pd.read_csv("https://drive.google.com/uc?&confirm=no_antivirus&id=1wv4xDpDEkImctyyMdZFWYlIOK4IDnUDP")
transactions = pd.read_csv("https://drive.google.com/uc?&confirm=no_antivirus&id=1ienvneTVRPXEGYK4n8OtfW_80_kySg6R")
creditScore = pd.read_csv("https://drive.google.com/uc?&confirm=no_antivirus&id=1i2YVzFXKPGzO2qQs9zNZzYws1aWLD0pS")
# To work with local files, uncomment these lines and write the local path of files:
#clients = pd.read_csv("C:/Users/DavidG/Documents/Datacfp/BASE DE DATOS/clients_table.txt")
#clientProducts = pd.read_csv("C:/Users/DavidG/Documents/Datacfp/BASE DE DATOS/products_table.txt")
#transactions = pd.read_csv("C:/Users/DavidG/Documents/Datacfp/BASE DE DATOS/transactions_table.txt")
#creditScore = pd.read_csv("C:/Users/DavidG/Documents/Datacfp/BASE DE DATOS/credit_score_table.txt")

## Desired Population
* Contracts from 2015 onwards.
* Operations in Italy were closed this year (2019).
* Take out clients with more than 75% of their info missing.
* Each client should only have one contract in the database, no more.
* A client to be eligible, it must have at least two years of information within the company (even if it canceled the product).

In [3]:
clients.head() # First, I see the general structure of the clients' dataset

,CustomerId,Surname,Geography,Gender,HasCrCard,IsActiveMember,EstimatedSalary,application_date,exit_date,birth_date
0,15745584,EIRLS,Germany,Female,0.0,1.0,0.00,2018-12-14,NaN,1997-09-18
1,14990118,MOLOCK,Italy,Male,1.0,0.0,121219.28,2019-01-08,NaN,1980-08-03
2,14733224,PAWLUCH,Italy,Female,1.0,1.0,159663.59,2012-08-01,2013-08-09,1977-08-19
3,14648573,NALLS,Spain,Male,1.0,0.0,140827.98,2019-06-19,NaN,1979-02-27
4,15365443,EBERLE,Italy,Male,1.0,0.0,35521.28,2014-01-26,2015-12-04,1972-12-21


In [4]:
n_entries = len(clients)
n_unique_clients = len(clients['CustomerId'].unique())
print(n_entries == n_unique_clients) # I check if each client has only one entry in the dataset, The result is false, I will try to drop duplicate entries.
print("{} entries are in excess".format(n_entries - n_unique_clients)) # This is the number of entries that I have in excess.
clients.drop_duplicates(inplace = True) # Drop duplicates entries
assert len(clients['CustomerId'].unique()) == len(clients) # Check if this process gives me the result I need. 
# It's works
print("I started with {} entries and I finished with {} unique entries". format(n_entries, n_unique_clients))

False
45000 entries are in excess
I started with 1545000 entries and I finished with 1500000 unique entries


In [5]:
clients['application_date'] = pd.to_datetime(clients['application_date']) # Then, I convert to datetime the 'application_date' column
clients = clients[(clients['application_date'] >= '2015') 
                  & (clients['application_date'] <= max(clients['application_date']) - pd.Timedelta('730d'))]
# I apply a filter with querying contracts from 2015 until the max date minus 730 days (2 years) Pandas will not support Y, M, and y values.
# It's better to use days and avoids future headaches.
print("After selecting only contracts from 2015 to 2 years before the last contract. The dataset has {} entries". format(len(clients)))

After selecting only contracts from 2015 to 2 years before the last contract. The dataset has 10000 entries


In [6]:
print(clients['Geography'].unique()) # After applying the filter, The dataset doesn't have clients from Italy. For this reason, It isn't necessary to apply a filter for Italy´s clients.
print(clients.apply(lambda x: sum(x.isna()), axis = 0)) # In this step, I check the clients who have lost information. Only in exit_date appears missing data.
# For this reason, I don't need to apply a filter to leave clients with 25% or more information.
print("The final number of clients after filtering is {}". format(len(clients)))

['Germany' 'France' 'Spain']
CustomerId             0
Surname                0
Geography              0
Gender                 0
HasCrCard              0
IsActiveMember         0
EstimatedSalary        0
application_date       0
exit_date           2817
birth_date             0
dtype: int64
The final number of clients after filtering is 10000


In [7]:
clients['birth_date'] = pd.to_datetime(clients['birth_date']) # Then, I convert to datetime the 'birth_date' column
clients['Age'] = (clients['application_date'] - clients['birth_date']).astype('timedelta64[Y]') # And I create the column age at the moment of application.
clients.reset_index(drop = True, inplace = True) # Finally I reset the index
# The clients' dataset is ready to use.

## Products dataset
This dataset is used to get the number of products per client at the moment of application.

In [8]:
print(sum(clientProducts.duplicated())) # First, I check if the clientProducts dataset has duplicate entries. The result is 0, There aren't duplicate entries.
products = clientProducts.groupby('CustomerId').agg({'Products': len}) # Then, I create a new dataset with the number of products of each client. I use groupby and agg to make it optimized.
clients = pd.merge(clients, products, how='left', on='CustomerId') # Finally, I merge the clients' dataset with the product dataset.

0


## Transactions dataset
This dataset is used to get the balance in the bank account at the moment of application.

In [9]:
print(transactions.dtypes) # First, I check if the columns' types are correct to manipulate it. The 'Value' column is float, It's correct to get the balance.
print(sum(transactions.duplicated())) # I check if the transaction dataset has duplicate entries. The result is 0, There aren't duplicate entries.
balance = transactions.groupby('CustomerId').agg({'Value': np.sum}) # Then, I create a new dataset with the value in the clients' accounts. I use groupby and agg to make it optimized.
clients = pd.merge(clients, balance, how='left', on='CustomerId') # Finally, I merge the clients' dataset with the balance dataset.

CustomerId       int64
Transaction     object
Value          float64
dtype: object
0


## Credit bureau scores
This dataset is used to get the score of the credit bureau at the moment of application.

In [10]:
creditScore['Date'] = pd.to_datetime(creditScore['Date']) # First, I convert 'Date' column to datetime format
# To search the credit score at the moment of application, I create the function 'get_score' to apply each row of clients' dataset
def get_score(row):
    client_scores = creditScore[creditScore['CustomerId'] == row['CustomerId']] # In the function, first I get the data of the client who I need
    try:
    # Then, I try to get the score with the comparison of the date of application and the date in credit data client
        score = client_scores[client_scores['Date'].dt.to_period('M') == row['application_date'].to_period('M')]['Score']
        return int(score)
    except:
    # If I don't find the value, I return NaN value
        return np.NaN

clients['Score'] = clients.apply(get_score, axis = 1) # I apply the function. Always Apply is better than a for loop ;)
print(sum(clients['Score'].isna())) # I check if the apply function doesn't find a value, The result is 0, all values are found.

0


# Statistics variables
In this section, I find the mean, standard deviation, maximum, and minimum of the age, products, balance, and score.

In [12]:
clients.rename(columns = {'Value': 'Balance'}, inplace = True) # First I rename the 'Value' column to a more explicit name
# Then, I select only columns that I need and find statistics variables with agg function.
clients[['Age', 'Products', 'Balance', 'Score']].agg({'Age': [np.mean, np.std, np.max, np.min], 
                                                      'Products': [np.mean, np.std, np.max, np.min],
                                                      'Balance': [np.mean, np.std, np.max, np.min],
                                                      'Score': [np.mean, np.std, np.max, np.min]})

,Age,Products,Balance,Score
mean,38.921800,1.530200,7.648589e+04,650.528800
std,10.487806,0.581654,6.239741e+04,96.653299
amax,92.000000,4.000000,2.508981e+05,850.000000
amin,18.000000,1.000000,-1.818989e-12,350.000000
